<a href="https://colab.research.google.com/github/RahulHipparkar/DM-Assignment-02-Automated-Dataset-Insight-Generator-LLM-Assisted-EDA-/blob/main/Assignment_02_Automated_Dataset_Insight_Generator_(LLM_Assisted_EDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

In our in-class activity, we practiced “getting to know your data” by loading a dataset, checking its structure, computing descriptive statistics, creating plots, and writing short interpretations about patterns and data quality. This assignment extends that workflow: you will build a Python system that can analyze any dataset a user uploads and automatically produce a clear set of useful data insights.

Your system will accept a CSV file as input (required). It may also accept an optional schema / data dictionary file that describes the columns (e.g., name, type, meaning, units, allowed values, missing-value codes). Your system must still run and produce results even if the schema file is not provided.

You have full access to GenAI tools, including cloud-based models (e.g., ChatGPT, Gemini, Claude) and local/open-source LLMs (e.g., Mistral, DeepSeek, Llama, or similar). You may use GenAI for brainstorming, code assistance, and generating narrative insight. However, you must document your GenAI usage by including a prompt/response log in your repository, and you remain responsible for verifying correctness (i.e., do not present unsupported or hallucinated conclusions as facts).

# Team Overview

**Team Members : Rahul Hipparkar, Himanshu Jain**

# Importing Libraries

In [33]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google import genai
import json


# Importing Datasets

In [34]:
!wget -q -O wine_red.csv "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

In [35]:
df_train = sns.load_dataset('tips')     # development dataset
df_unseen = pd.read_csv("wine_red.csv", sep=";")  # unseen dataset

# Intializing GEMINI

In [36]:
from google.colab import userdata
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

In [37]:
os.makedirs("logs", exist_ok=True)

In [38]:
models = client.models.list()
for m in models:
    # Some SDK versions expose fields slightly differently; print what you have
    print(getattr(m, "name", None), getattr(m, "supported_generation_methods", None))


models/gemini-2.5-flash None
models/gemini-2.5-pro None
models/gemini-2.0-flash None
models/gemini-2.0-flash-001 None
models/gemini-2.0-flash-exp-image-generation None
models/gemini-2.0-flash-lite-001 None
models/gemini-2.0-flash-lite None
models/gemini-exp-1206 None
models/gemini-2.5-flash-preview-tts None
models/gemini-2.5-pro-preview-tts None
models/gemma-3-1b-it None
models/gemma-3-4b-it None
models/gemma-3-12b-it None
models/gemma-3-27b-it None
models/gemma-3n-e4b-it None
models/gemma-3n-e2b-it None
models/gemini-flash-latest None
models/gemini-flash-lite-latest None
models/gemini-pro-latest None
models/gemini-2.5-flash-lite None
models/gemini-2.5-flash-image None
models/gemini-2.5-flash-preview-09-2025 None
models/gemini-2.5-flash-lite-preview-09-2025 None
models/gemini-3-pro-preview None
models/gemini-3-flash-preview None
models/gemini-3-pro-image-preview None
models/nano-banana-pro-preview None
models/gemini-robotics-er-1.5-preview None
models/gemini-2.5-computer-use-preview-10

In [39]:
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="Say hello in one sentence."
)
print(response.text)


Hello, I hope you are having a wonderful day!


# Function to utilize Gemini to do EDA and generate insights

In [40]:

MISSING_CODES = [-999, 9999, -1, "NA", "N/A", "na", "null", "NULL", "unknown", "Unknown", ""]

def run_eda_with_gemini(df, run_name, model="gemini-flash-latest"):
    outdir = Path("output") / run_name
    plots_dir = outdir / "plots"
    tables_dir = outdir / "tables"
    outdir.mkdir(parents=True, exist_ok=True)
    plots_dir.mkdir(parents=True, exist_ok=True)
    tables_dir.mkdir(parents=True, exist_ok=True)

    # Clean missing codes
    dfx = df.copy()
    dfx.replace(MISSING_CODES, np.nan, inplace=True)

    #Overview + quality
    overview = {
        "rows": int(dfx.shape[0]),
        "cols": int(dfx.shape[1]),
        "columns": dfx.columns.tolist(),
        "inferred_types": {c: str(dfx[c].dtype) for c in dfx.columns},
        "duplicates": int(dfx.duplicated().sum()),
    }

    missing_tbl = (
        dfx.isna().sum().to_frame("missing_count")
        .assign(missing_pct=lambda x: (x["missing_count"]/len(dfx)*100).round(2))
        .sort_values("missing_pct", ascending=False)
        .reset_index().rename(columns={"index":"column"})
    )
    missing_tbl.to_csv(tables_dir / "missing_summary.csv", index=False)

    nunique = dfx.nunique(dropna=True)
    one_value_cols = nunique[nunique <= 1].index.tolist()
    high_missing_cols = missing_tbl.loc[missing_tbl["missing_pct"] > 40, "column"].tolist()

    quality = {"one_value_cols": one_value_cols, "high_missing_cols": high_missing_cols}

    # Categorical summary (1 column)
    cat_cols = dfx.select_dtypes(exclude="number").columns.tolist()
    cat_col, cat_tbl = None, None
    if cat_cols:
        # choose a moderate-cardinality categorical column
        candidates = []
        for c in cat_cols:
            u = dfx[c].nunique(dropna=True)
            if 2 <= u <= 50:
                candidates.append((abs(u-10), u, c))
        cat_col = sorted(candidates)[0][2] if candidates else cat_cols[0]
        vc = dfx[cat_col].astype("string").value_counts(dropna=True)
        total = dfx[cat_col].notna().sum()
        cat_tbl = (
            vc.head(20).to_frame("count")
            .assign(pct=lambda x: (x["count"]/total*100).round(2))
            .reset_index().rename(columns={"index":"value"})
        )
        cat_tbl.to_csv(tables_dir / "categorical_freq.csv", index=False)

    # Numeric stats (≥2 if available) + outliers (IQR)
    num_cols = dfx.select_dtypes(include="number").columns.tolist()
    num_rows = []
    for c in num_cols:
        s = dfx[c].dropna()
        if len(s) == 0:
            continue
        q1, q3 = s.quantile([0.25, 0.75])
        iqr = float(q3 - q1)
        lo, hi = float(q1 - 1.5*iqr), float(q3 + 1.5*iqr)
        outlier_pct = float(((s < lo) | (s > hi)).mean() * 100)
        mode_val = s.mode()
        mode_val = float(mode_val.iloc[0]) if len(mode_val) else np.nan

        num_rows.append({
            "column": c,
            "min": float(s.min()), "max": float(s.max()),
            "mean": float(s.mean()), "median": float(s.median()),
            "mode": mode_val,
            "std": float(s.std(ddof=1)) if len(s) > 1 else 0.0,
            "iqr": iqr,
            "outlier_low": lo, "outlier_high": hi,
            "outlier_pct": round(outlier_pct, 2),
            "n_nonnull": int(len(s)),
        })

    num_tbl = pd.DataFrame(num_rows).sort_values("n_nonnull", ascending=False)
    num_tbl.to_csv(tables_dir / "numeric_summary.csv", index=False)

    # Make ≥5 plots
    plots = []
    def savefig(path):
        plt.savefig(path, bbox_inches="tight"); plt.close()
        plots.append(str(path))

    # 1 histogram
    if num_cols:
        c = num_cols[0]
        plt.figure()
        sns.histplot(dfx[c].dropna(), bins=30)
        plt.title(f"Histogram: {c}"); plt.xlabel(c); plt.ylabel("Count")
        savefig(plots_dir / f"hist_{c}.png")

    # 2 boxplot
    if len(num_cols) > 1:
        c = num_cols[1]
        plt.figure()
        sns.boxplot(x=dfx[c])
        plt.title(f"Boxplot: {c}"); plt.xlabel(c); plt.ylabel("Value")
        savefig(plots_dir / f"box_{c}.png")

    # 3 bar chart
    if cat_cols:
        c = cat_cols[0]
        vc = dfx[c].astype("string").value_counts(dropna=True).head(10)
        plt.figure()
        vc.plot(kind="bar")
        plt.title(f"Top 10 Categories: {c}"); plt.xlabel(c); plt.ylabel("Count")
        savefig(plots_dir / f"bar_{c}.png")

    # 4 scatter
    if len(num_cols) >= 2:
        x, y = num_cols[0], num_cols[1]
        plt.figure()
        sns.scatterplot(x=dfx[x], y=dfx[y])
        plt.title(f"Scatter: {x} vs {y}"); plt.xlabel(x); plt.ylabel(y)
        savefig(plots_dir / f"scatter_{x}_vs_{y}.png")

    # 5 correlation heatmap
    if len(num_cols) >= 2:
        plt.figure(figsize=(7,6))
        sns.heatmap(dfx[num_cols].corr(numeric_only=True), annot=False)
        plt.title("Correlation Heatmap (Numeric Columns)")
        plt.xlabel("Columns"); plt.ylabel("Columns")
        savefig(plots_dir / "corr_heatmap.png")

    # fallback if <5
    if len(plots) < 5:
        miss = dfx.isna().mean().mul(100).sort_values(ascending=False).head(20)
        plt.figure(figsize=(8,4))
        miss.plot(kind="bar")
        plt.title("Top Missingness by Column (%)")
        plt.xlabel("Column"); plt.ylabel("Missing %")
        savefig(plots_dir / "missingness_top.png")

    # Gemini insights from computed facts only
    facts_for_gemini = {
        "overview": overview,
        "quality_checks": quality,
        "missing_top10": missing_tbl.head(10).to_dict(orient="records"),
        "categorical_summary": None if cat_tbl is None else {
            "column": cat_col,
            "top_values": cat_tbl.head(10).to_dict(orient="records"),
        },
        "numeric_summary_top": num_tbl.head(10).to_dict(orient="records"),
        "plots_saved": plots,
    }

    client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
    prompt = f"""
You are an EDA assistant.

STRICT RULES:
- Use ONLY the numbers/columns in FACTS_JSON.
- Do NOT invent any values.
- Output:
  1) 5–10 bullet insights (each must mention a specific column + a number from the facts)
  2) 1 short limitations/bias note (missingness, sampling, coverage, etc.)

FACTS_JSON:
{json.dumps(facts_for_gemini, indent=2)}
"""
    resp = client.models.generate_content(model=model, contents=prompt)
    insights_text = resp.text

    # Write report.md
    report = []
    report.append(f"# Automated EDA Report — {run_name}\n")
    report.append("## Dataset Overview\n")
    report.append(f"- Rows: **{overview['rows']}**\n- Columns: **{overview['cols']}**\n- Duplicates: **{overview['duplicates']}**\n")

    report.append("\n### Inferred Types (first 15)\n")
    for c in overview["columns"][:15]:
        report.append(f"- {c}: {overview['inferred_types'][c]}\n")

    report.append("\n### Missing Value Summary (Top 10)\n")
    report.append(missing_tbl.head(10).to_markdown(index=False))

    report.append("\n## Basic Data Quality Checks\n")
    report.append(f"- Columns with one value: {quality['one_value_cols']}\n")
    report.append(f"- High-missing columns (>40%): {quality['high_missing_cols']}\n")

    report.append("\n## Descriptive Statistics\n")
    if cat_tbl is not None:
        report.append(f"### Categorical: `{cat_col}` (Top values)\n")
        report.append(cat_tbl.head(10).to_markdown(index=False))
    else:
        report.append("No categorical columns detected.\n")

    report.append("\n### Numeric (Top rows)\n")
    if not num_tbl.empty:
        report.append(num_tbl.head(10).to_markdown(index=False))
    else:
        report.append("No numeric columns detected.\n")

    report.append("\n## Visualizations (saved files)\n")
    for p in plots:
        report.append(f"- {p}\n")

    report.append("\n## Insights + Limitations (Gemini)\n")
    report.append(insights_text)

    (outdir / "report.md").write_text("\n".join(report), encoding="utf-8")

    # ---- (H) Save log for this run
    log_path = Path("logs") / f"genai_log_{run_name}.md"
    log_path.write_text(
        "## Tool used\nGemini API via google-genai\n\n"
        f"## Prompt\n```text\n{prompt[:3500]}\n```\n\n"
        f"## Response\n```text\n{insights_text[:3500]}\n```\n",
        encoding="utf-8"
    )

    return str(outdir / "report.md")


# Inference

In [41]:
dev_report = run_eda_with_gemini(df_train, run_name="dev_training_dataset")
dev_report

'output/dev_training_dataset/report.md'

In [42]:
unseen_report = run_eda_with_gemini(df_unseen, run_name="unseen_inference_dataset")
unseen_report

'output/unseen_inference_dataset/report.md'